### 01.문제 정의

-> 10개의 아울렛 매장에서 1,500여개의 제품에 대한 판매 데이터를 수집했다. 예측 모델을 만들고 아울렛 특정 매장에서 각 제품의 판매 금액을 예측하시오
- 평가 기준은 RMSE
- label(target)은 판매금액(Item_Outlet_Sale)
- 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명: pred, 1개)

 ### 02. 라이브러리 및 데이터 불러오기

In [3]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

 ### 03. 탐색적 데이터 분석

In [4]:
train.shape, test.shape

((6818, 12), (1705, 11))

In [5]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


In [12]:
train.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDW26,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


In [13]:
test.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,FDG33,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


In [14]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [15]:
test.isnull().sum()

Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

 ### 04. 데이터 전처리

In [17]:
list(train.columns[train.dtypes==object])

['Item_Identifier',
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [18]:
cols = [
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']
cols

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [19]:
# 인코딩을 위해 train, test를 병합한다.

In [ ]:
# 합쳐서 진행하지 않을 경우 탐색적 데이터 분석 과정에서 카테고리 종류가 다른 경우로 학습시 에러가 발생할 수 있다.

In [20]:
target = train.pop('Item_Outlet_Sales')

In [21]:
print(train.shape, test.shape)

(6818, 11) (1705, 11)


In [22]:
df = pd.concat([train, test])

In [23]:
print(df.shape)

(8523, 11)


In [24]:
#사이킷런에서 제공하는 LabelEncoder로 레이블 인코딩을 한다. train, test가 합쳐진 df 데이터만 레이블 인코딩하면 된다.

In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols : 
    df[col] = le.fit_transform(df[col])

In [28]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...,...
6813,NCC54,17.750,Low Fat,0.097863,Health and Hygiene,239.3196,OUT049,1999,Medium,Tier 1,Supermarket Type1
6814,NCF19,13.000,Low Fat,0.035252,Household,48.0034,OUT018,2009,Medium,Tier 3,Supermarket Type2
6815,FDU11,4.785,Low Fat,0.092576,Breads,121.0098,OUT035,2004,Small,Tier 2,Supermarket Type1
6816,FDK14,6.980,Low Fat,0.041189,Canned,82.8934,OUT045,2002,NaN,Tier 2,Supermarket Type1


In [29]:
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()
train.shape, test.shape

((6818, 11), (1705, 11))

In [30]:
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])

In [31]:
print(train.shape, test.shape)
train.drop('Item_Identifier', axis=1, inplace=True)
test.drop('Item_Identifier', axis=1, inplace=True)
print(train.shape, test.shape)

(6818, 11) (1705, 11)
(6818, 10) (1705, 10)


 ### 05. 검증 데이터 나누기

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 10), (1364, 10), (5454,), (1364,))

 ### 06. 머신러닝 학습 및 평가

In [34]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
try:
    from sklearn.metrics import root_mean_squared_error
except ImportError:
    print("Using older sklearn version for RMSE")
    root_mean_squared_error = None

Using older sklearn version for RMSE


#### (1) 선형 회귀

In [35]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print("MSE : ", result)

result = mean_absolute_error(y_val, y_pred)
print("MAE : ", result)

result = r2_score(y_val, y_pred)
print("R2 : ", result)


MSE :  1282923.072983391
MAE :  865.1968401416278
R2 :  0.5058168396924837


In [ ]:
#### (2) 랜덤 포레스트

In [36]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print("MSE : ", result)

result = mean_absolute_error(y_val, y_pred)
print("MAE : ", result)

result = r2_score(y_val, y_pred)
print("R2 : ", result)


MSE :  1101802.8117346708
MAE :  747.2971172067448
R2 :  0.5755845326933362


In [42]:
pip install --upgrade pip


  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/44/3c/d717024885424591d5376220b5e836c2d5293ce2011523c9de23ff7bf068/pip-25.3-py3-none-any.whl.metadata
   ---------------------------------------- 1.8/1.8 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1


In [43]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [45]:
import lightgbm as lgb
model = lgb.LGBMRegressor(verbose=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print("MSE : ", result)

result = mean_absolute_error(y_val, y_pred)
print("MAE : ", result)

result = r2_score(y_val, y_pred)
print("R2 : ", result)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000231 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 787
[LightGBM] [Info] Number of data points in the train set: 5454, number of used features: 10
[LightGBM] [Info] Start training from score 2202.546849
MSE :  1115654.3482227568
MAE :  736.6367966578568
R2 :  0.5702489079618556


In [ ]:
 ### 07. 예측 및 결과 파일 생성

In [47]:
pred = model.predict(test)

In [48]:
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

In [49]:
pd.read_csv("result.csv")

,pred
0,1226.505042
1,830.189210
2,1788.766443
3,1644.510784
4,2831.798098
...,...
1700,285.648607
1701,481.591492
1702,3666.348761
1703,990.651255
